In [1]:
import backtesting
from ..utils import tradingUtils as ut  # Relative import of the module
from ..utils.tradingUtils import OurStrategy  # Relative import of specific class
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from bokeh.io import output_notebook
output_notebook()
backtesting.set_bokeh_output(notebook=True)  # Set Bokeh for notebook output
import numpy as np


C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: attempted relative import with no known parent package

## Doji Pattern

In [17]:
tsla_data = ut.tsla_data

In [18]:
def doji_pattern_policy(data, body):
    result = pd.Series(0, index=data.index)

    close = data['Close'].values
    open_ = data['Open'].values

    for i in range(2, len(data)):  # Start from the 3rd data point
        # Bullish Doji pattern
        if (
            (close[i] > open_[i]) and 
            (close[i] > close[i - 1]) and  
            (abs(close[i - 1] - open_[i - 1]) < body) and  
            (close[i - 2] < open_[i - 2]) 
        ):
            result.iloc[i] = 1  # Buy signal for the next day

        # Bearish Doji pattern
        elif (
            (close[i] < open_[i]) and  # Current candle bearish
            (close[i] < close[i - 1]) and  # Current close lower than previous close
            (abs(close[i - 1] - open_[i - 1]) < body) and  # Previous candle is a Doji
            (close[i - 2] > open_[i - 2])  # Two candles ago was bullish
        ):
            result.iloc[i] = -1  # Sell signal for the next day

    return result


In [19]:
# Gerando os sinais que dão match com a política
policy = doji_pattern_policy(tsla_data, body=0.1)

# Adicionando uma coluna sinal
tsla_data["Signal"] = policy
# Definindo os pontos de saída
ut.exit_points(tsla_data, policy, 5, 1)

# Gerando o modelo
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()
bt.plot()

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p2356', ...)

In [20]:
print(stats)

Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                   13.928672
Equity Final [$]                 41222.725877
Equity Peak [$]                  41222.725877
Return [%]                         312.227259
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                   10.542977
Volatility (Ann.) [%]               16.034003
Sharpe Ratio                         0.657539
Sortino Ratio                        1.189473
Calmar Ratio                         0.450054
Max. Drawdown [%]                  -23.426017
Avg. Drawdown [%]                   -4.320687
Max. Drawdown Duration      421 days 00:00:00
Avg. Drawdown Duration      111 days 00:00:00
# Trades                                  201
Win Rate [%]                         57.21393
Best Trade [%]                      16.103145
Worst Trade [%]                      -9.73276
Avg. Trade [%]                    

## Harami Pattern

In [21]:
def harami_pattern_policy(data):
    result = pd.Series(0, index=data.index)

    open_ = data['Open'].values
    close = data['Close'].values
    high = data['High'].values
    low = data['Low'].values

    for i in range(2, len(data)):
        # Bullish Harami pattern
        if (
            (close[i] > open_[i]) and  
            (close[i] < open_[i - 1]) and  
            (open_[i] > close[i - 1]) and 
            (high[i] < high[i - 1]) and 
            (low[i] > low[i - 1]) and  
            (close[i - 1] < open_[i - 1]) and  
            (close[i - 2] < open_[i - 2])  
        ):
            result.iloc[i] = 1  # Buy signal

        # Bearish Harami pattern
        elif (
            (close[i] < open_[i]) and  
            (close[i] > open_[i - 1]) and  
            (open_[i] < close[i - 1]) and  
            (high[i] < high[i - 1]) and  
            (low[i] > low[i - 1]) and  
            (close[i - 1] > open_[i - 1]) and  
            (close[i - 2] > open_[i - 2])  
        ):
            result.iloc[i] = -1  # Sell signal

    return result


In [22]:
# Gerando os sinais que dão match com a política
policy = harami_pattern_policy(tsla_data)

# Adicionando uma coluna sinal
tsla_data["Signal"] = policy
# Definindo os pontos de saída
ut.exit_points(tsla_data, policy, 5, 1)

# Gerando o modelo
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()
bt.plot()

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p2702', ...)

In [23]:
print(stats)

Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                    2.190396
Equity Final [$]                 10302.215365
Equity Peak [$]                  10777.516664
Return [%]                           3.022154
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                    0.210922
Volatility (Ann.) [%]                6.733331
Sharpe Ratio                         0.031325
Sortino Ratio                        0.055032
Calmar Ratio                         0.010725
Max. Drawdown [%]                  -19.665982
Avg. Drawdown [%]                  -12.875305
Max. Drawdown Duration     3708 days 00:00:00
Avg. Drawdown Duration     2313 days 00:00:00
# Trades                                   39
Win Rate [%]                        41.025641
Best Trade [%]                      17.003709
Worst Trade [%]                     -6.861963
Avg. Trade [%]                    

## On the neck

In [ ]:
def on_the_neck_pattern_policy(data, body_range):
    result = pd.Series(0, index=data.index)

    open_ = data['Open'].values
    close = data['Close'].values
    high = data['High'].values
    low = data['Low'].values

    for i in range(1, len(data)):
        # Bearish On The Neck pattern
        if (
            (close[i - 1] < open_[i - 1]) and  
            (close[i] == low[i - 1]) and  
            (close[i] > open_[i]) and  
            (abs(close[i] - open_[i]) <= body_range)  
        ):
            result.iloc[i] = -1  # Sell signal

        # Bullish On The Neck pattern 
        elif (
            (close[i - 1] > open_[i - 1]) and  
            (close[i] == high[i - 1]) and  
            (close[i] < open_[i]) and  
            (abs(close[i] - open_[i]) <= body_range)  
        ):
            result.iloc[i] = 1  # Buy signal

    return result
